Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/deployment/deploy-to-local/register-model-deploy-local.png)

# Register model and deploy locally

This example shows how to deploy a web service in step-by-step fashion:

 1. Register model
 2. Deploy the image as a web service in a local Docker container.
 3. Quickly test changes to your entry script by reloading the local service.
 4. Optionally, you can also make changes to model, conda or extra_docker_file_steps and update local service

## Prerequisites
If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you go through the [configuration](../../../configuration.ipynb) Notebook first if you haven't.

In [ ]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

## Initialize Workspace

Initialize a workspace object from persisted configuration.

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

## Create trained model

For this example, we will train a small model on scikit-learn's [diabetes dataset](https://scikit-learn.org/stable/datasets/toy_dataset.html#diabetes-dataset). 

In [ ]:
import joblib

from sklearn.datasets import load_diabetes
from sklearn.linear_model import Ridge

dataset_x, dataset_y = load_diabetes(return_X_y=True)

sk_model = Ridge().fit(dataset_x, dataset_y)

joblib.dump(sk_model, "sklearn_regression_model.pkl")

## Register Model

Here we are registering the serialized file `sklearn_regression_model.pkl` in the current directory as a model with the name `sklearn_regression_model` in the workspace.

You can add tags and descriptions to your models. Using tags, you can track useful information such as the name and version of the machine learning library used to train the model, framework, category, target customer etc. Note that tags must be alphanumeric.

In [ ]:
from azureml.core.model import Model

model = Model.register(model_path="sklearn_regression_model.pkl",
                       model_name="sklearn_regression_model",
                       tags={'area': "diabetes", 'type': "regression"},
                       description="Ridge regression model to predict diabetes",
                       workspace=ws)

## Create Environment

In [ ]:
import sklearn

from azureml.core.environment import Environment

environment = Environment("LocalDeploy")
environment.python.conda_dependencies.add_pip_package("inference-schema[numpy-support]")
environment.python.conda_dependencies.add_pip_package("joblib")
environment.python.conda_dependencies.add_pip_package("scikit-learn=={}".format(sklearn.__version__))

## Provide the Scoring Script

This Python script handles the model execution inside the service container. The `init()` method loads the model file, and `run(data)` is called for every input to the service.

In [ ]:
%%writefile score.py
import joblib
import json
import numpy as np
import os

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType

def init():
    global model
    # AZUREML_MODEL_DIR is an environment variable created during deployment.
    # It is the path to the model folder (./azureml-models/$MODEL_NAME/$VERSION)
    # For multiple models, it points to the folder containing all deployed models (./azureml-models)
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'sklearn_regression_model.pkl')
    # Deserialize the model file back into a sklearn model.
    model = joblib.load(model_path)

input_sample = np.array([[10.0, 9.0, 8.0, 7.0, 6.0, 5.0, 4.0, 3.0, 2.0, 1.0]])
output_sample = np.array([3726.995])

@input_schema('data', NumpyParameterType(input_sample))
@output_schema(NumpyParameterType(output_sample))
def run(data):
    try:
        result = model.predict(data)
        # You can return any JSON-serializable object.
        return result.tolist()
    except Exception as e:
        error = str(e)
        return error

## Create Inference Configuration

In [ ]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script="score.py",
                                   environment=environment)

## Deploy Model as a Local Docker Web Service

*Make sure you have Docker installed and running.*

Note that the service creation can take few minutes.

NOTE:

The Docker image runs as a Linux container. If you are running Docker for Windows, you need to ensure the Linux Engine is running:

    # PowerShell command to switch to Linux engine
    & 'C:\Program Files\Docker\Docker\DockerCli.exe' -SwitchLinuxEngine

In [ ]:
from azureml.core.webservice import LocalWebservice

# This is optional, if not provided Docker will choose a random unused port.
deployment_config = LocalWebservice.deploy_configuration(port=6789)

local_service = Model.deploy(ws, "test", [model], inference_config, deployment_config)

local_service.wait_for_deployment()

In [ ]:
print('Local service port: {}'.format(local_service.port))

## Check Status and Get Container Logs


In [ ]:
print(local_service.get_logs())

## Test Web Service

Call the web service with some input data to get a prediction.

In [ ]:
import json

sample_input = json.dumps({
    'data': dataset_x[0:2].tolist()
})

local_service.run(sample_input)

## Reload Service

You can update your score.py file and then call `reload()` to quickly restart the service. This will only reload your execution script and dependency files, it will not rebuild the underlying Docker image. As a result, `reload()` is fast, but if you do need to rebuild the image -- to add a new Conda or pip package, for instance -- you will have to call `update()`, instead (see below).

In [ ]:
%%writefile score.py
import joblib
import json
import numpy as np
import os

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType

def init():
    global model
    # AZUREML_MODEL_DIR is an environment variable created during deployment.
    # It is the path to the model folder (./azureml-models/$MODEL_NAME/$VERSION)
    # For multiple models, it points to the folder containing all deployed models (./azureml-models)
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'sklearn_regression_model.pkl')
    # Deserialize the model file back into a sklearn model.
    model = joblib.load(model_path)

input_sample = np.array([[10.0, 9.0, 8.0, 7.0, 6.0, 5.0, 4.0, 3.0, 2.0, 1.0]])
output_sample = np.array([3726.995])

@input_schema('data', NumpyParameterType(input_sample))
@output_schema(NumpyParameterType(output_sample))
def run(data):
    try:
        result = model.predict(data)
        # You can return any JSON-serializable object.
        return 'Hello from the updated score.py: ' + str(result.tolist())
    except Exception as e:
        error = str(e)
        return error

In [ ]:
local_service.reload()
print("--------------------------------------------------------------")

# After calling reload(), run() will return the updated message.
local_service.run(sample_input)

## Update Service

If you want to change your model(s), Conda dependencies or deployment configuration, call `update()` to rebuild the Docker image.


In [ ]:
local_service.update(models=[model],
                     inference_config=inference_config,
                     deployment_config=deployment_config)


## Deploy model to AKS cluster based on the LocalWebservice's configuration.

In [ ]:
# This is a one time setup for AKS Cluster. You can reuse this cluster for multiple deployments after it has been created. If you delete the cluster or the resource group that contains it, then you would have to recreate it.
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your AKS cluster
aks_name = 'my-aks-9' 

# Verify the cluster does not exist already
try:
    aks_target = ComputeTarget(workspace=ws, name=aks_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # Use the default configuration (can also provide parameters to customize)
    prov_config = AksCompute.provisioning_configuration()

    # Create the cluster
    aks_target = ComputeTarget.create(workspace = ws, 
                                      name = aks_name, 
                                      provisioning_configuration = prov_config)

if aks_target.get_status() != "Succeeded":
    aks_target.wait_for_completion(show_output=True)

In [ ]:
from azureml.core.webservice import AksWebservice
# Set the web service configuration (using default here)
aks_config = AksWebservice.deploy_configuration()

# # Enable token auth and disable (key) auth on the webservice
# aks_config = AksWebservice.deploy_configuration(token_auth_enabled=True, auth_enabled=False)


In [ ]:
%%time
aks_service_name ='aks-service-1'

aks_service = local_service.deploy_to_cloud(name=aks_service_name,
                                            deployment_config=aks_config,
                                            deployment_target=aks_target)

aks_service.wait_for_deployment(show_output = True)
print(aks_service.state)

In [ ]:
# Test aks service

sample_input = json.dumps({
    'data': dataset_x[0:2].tolist()
})

aks_service.run(sample_input)

In [ ]:
# Delete the service if not needed.
aks_service.delete()

## Delete Service

In [ ]:
local_service.delete()